In [2]:
import scanpy as sc
from matplotlib.pyplot import rc_context
sc.set_figure_params(dpi=300, color_map="viridis_r")
sc.settings.verbosity = 0
sc.logging.print_header()

scanpy==1.10.1 anndata==0.10.5.post1 umap==0.5.5 numpy==1.25.0 scipy==1.11.4 pandas==2.0.3 scikit-learn==1.4.2 statsmodels==0.13.2 igraph==0.9.10 pynndescent==0.5.8


In [13]:
import anndata as ad
import pooch
import h5py
import numpy as np

In [28]:
### 在R中使用下面的代码对seurat对象进行更改：
# sample1[["RNA3"]] <- as(object = sample1[["RNA"]], Class = "Assay")
# DefaultAssay(sample1) <- "RNA3"
# sample1[["RNA"]] <- NULL
# sample1 <- RenameAssays(sample1, RNA3 = 'RNA')
# save_file <- "sample1.h5Seurat"
# SaveH5Seurat(sample1, filename = save_file, overwrite = TRUE)
# Convert(save_file, dest = "h5ad", overwrite = TRUE)
    
adata = sc.read_h5ad('./scanalysis/pubmed2.h5ad')

In [31]:
marker_genes_dict = {
    "C1-NSC group journals":["CNSsubjournal"],
    "C2-2023 CAS Rank": ["Chinese-Academy-of-Science", "University-of-Chinese-Academy-of-Science"],
    "C3-Early warning journals in medicine": ["YuJing"],
    "C4-Highcited author score": ["HighCitedAuthor"],
    "C5-Predatory journals 1": ["LveduoJournal"],
    "C6-Agricultural university": ["Jiangxi-Agricultural-University","Shenyang-Agricultural-University"],
    "C7-Affiliation foreign": ["Aff-Foreign"],
    "C8-Predatory journals with early warning 1": ["YuJing","LveduoJournal","Chinese-Academy-of-Science"],
    "C9-Predatory journals 2": ["LveduoJournal","North-University-of-China","Xian-Jiaotong-University"],
    "C10-Predatory journals 3": ["LveduoJournal"],
    "C11-Early warning journals in Qinghua university": ["YuJing","Tsinghua-University"],   
    "C12-Predatory journals with affiliation foreign": ["LveduoJournal","Aff-Foreign"],
    "C13-Predatory journals with early warning 2": ["LveduoJournal","Nanjing-forestry-University","Northeast-forestry-University"],
    "C14-Predatory journals in Zheda university": ["LveduoJournal","Zhejiang-University","Zhejiang-University-School-of-Medicine"],
    "C15-Early warning journals in other institution": ["YuJing","Shandong-University"],
}

In [60]:
cluster2annotation = {
    "0": "C1-NSC group journals",
    "1": "C1-NSC group journals",
    "2": "C2-2023 CAS Rank",
    "3": "C3-Early warning journals in medicine",
    "4": "C4-Highcited author score",
    "7": "C4-Highcited author score",
    "11": "C4-Highcited author score",
    "16": "C4-Highcited author score",
    "5": "C5-Predatory journals 1",
    "9": "C5-Predatory journals 1",
    "19": "C5-Predatory journals 1",
    "22": "C5-Predatory journals 1",
    "6": "C8-Predatory journals with early warning 1",
    "12": "C8-Predatory journals with early warning 1",
    "17": "C8-Predatory journals with early warning 1",
    "20": "C8-Predatory journals with early warning 1",
    "24": "C8-Predatory journals with early warning 1",
    "26": "C8-Predatory journals with early warning 1",
    "8": "C6-Agricultural university",
    "29": "C6-Agricultural university",
    "10": "C7-Affiliation foreign",
    "13": "C9-Predatory journals 2",
    "28": "C9-Predatory journals 2",
    "14": "C10-Predatory journals 3",
    "15": "C11-Early warning journals in Qinghua university",
    "18": "C12-Predatory journals with affiliation foreign",
    "21": "C13-Predatory journals with early warning 2",
    "23": "C14-Predatory journals in Zheda university",
    "25": "C15-Early warning journals in other institution",
    "27": "C15-Early warning journals in other institution",
}


In [61]:
adata.obs["cell type"] = adata.obs["seurat_clusters"].map(cluster2annotation).astype("category")

In [55]:
sc.tl.leiden(
    adata,
    key_added="seurat_clusters",
    resolution=0.5,
    n_iterations=2,
    flavor="igraph",
    directed=False,
)

In [53]:
sc.pp.neighbors(adata, n_neighbors=15, use_rep='X_pca')

/home/chench/anaconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [54]:
sc.tl.leiden(adata)

/tmp/ipykernel_1201/3467744858.py:1: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata)


In [86]:
adata

AnnData object with n_obs × n_vars = 50216 × 919
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'RNA_snn_res.0.3', 'seurat_clusters', 'celltype', 'group', 'Year', 'Affiliation', 'City', 'Province', 'papertype', 'leiden', 'cell type'
    var: 'vf_vst_counts_mean', 'vf_vst_counts_variance', 'vf_vst_counts_variance.expected', 'vf_vst_counts_variance.standardized', 'vf_vst_counts_variable', 'vf_vst_counts_rank', 'var.features', 'var.features.rank'
    uns: 'dendrogram_seurat_clusters', 'neighbors', 'leiden', 'seurat_clusters', 'seurat_clusters_colors', 'dendrogram_cell type', 'cell type_colors'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [87]:
adata.layers["scaled"] = sc.pp.scale(adata, copy=True).X

/home/chench/anaconda3/lib/python3.9/site-packages/scanpy/preprocessing/_scale.py:167: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt(var)
